<a href="https://colab.research.google.com/github/carlosOtalora16/Prediccion-de-valor-en-el-mercado-para-futbolistas/blob/main/Procesamiento_y_verificaci%C3%B3n_de_requisitos_Proyecto_I_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLamado de librerias y se añade la URL del dataset en formato csv el cual esta alojado en el repositorio github.

Esta celda debe ser ejecutada con prioridad para poder importar las librerias y hacer el llamdado del dataset en formato csv desde un link del repositorio de github del proyecto, ya que es primordial que las librerias esten importadas y el dataset descargado para no tener problemas al ejecutar las demás celdas.

#NOTA
El usuario no tendrá necesidad de buscar y descargar en su equipo para luego importar al notebook de colab ya que con esta primera celda la descarga del dataset del proyecto se hace directamente desde el repositorio de github del proyecto sin que el usuario tenga que hacer un paso adicional más que ejecutar esta primera celda. 

In [1]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/carlosOtalora16/Prediccion-de-valor-en-el-mercado-para-futbolistas/main/2021.csv'
'--------------------------------------------------------------------------------------------------------------'
#Importa archivo 2021.csv con su URL de github 
Datos_jugadores_fifa21_completo = pd.read_csv(url)

# Arreglos en el dataset.
Para hacer más sencillo el tratamiento y procesado de los datos se hace la eliminación de filas, dejando 5000 que es el mínimo para cumplir con las exigencias del proyecto, además, se eliminan columnas que no aportaran significativamente en el modelo de predicción del proyecto.

Tambien se establece la columna 'Name' como columna de indice para que sea mas sencillo tener informacion del dataset usando como indice el nombre de cada jugador. Se corrige el nombre de una columna que estaba mal escrita.

In [2]:
Datos_jugadores_fifa21 = Datos_jugadores_fifa21_completo.drop(range(5000,19117),axis=0).drop(['Specialities'
,'Traits','Club_JoinedClub','FKAcc','Aggression','StandTackle','SlideTackle','Club_ContractLength'
,'BirthDate','Wage','Nation_KitNumber','Club_KitNumber','Marking','Reactions','AttPosition','Weight','ID'],
axis=1).set_index('Name').rename(columns={'Natinality':'Nationality'})



# Tamaño del dataset y verificación de requisistos.
Se almacena el tamaño del dataset para saber el número de columnas categóricas que este debe tener para cumplir con los requisitos mínimos y luego saber cuantas columnas categóricas tiene el dataset.


In [3]:
Tamaño_Datos = Datos_jugadores_fifa21.shape #Arroja una tupla donde esta primero el numero de filas y luego las columnas
Numero_columnas_categoricas = int((Tamaño_Datos[1]*0.1)) #Se usa int para aproximar al numero entero mas cercano
#El dataset debe tener al menos 4 columnas de datos categoricos para cumplir la exigencia 

print(f'El dataset debe tener un minimo de {Numero_columnas_categoricas} columnas categoricas para cumplir con el requisito del proyecto.')
Columnas_categoricas = 0

for i in Datos_jugadores_fifa21.columns:
  
  if Datos_jugadores_fifa21[i].dtype =='object':
    Columnas_categoricas += 1

#Hay 9 columnas categoricas, se cumple con la exigencia 
Nombre_columnas_categoricas = [j for j in Datos_jugadores_fifa21.columns if Datos_jugadores_fifa21[j].dtype==object]
#List comprehension para saber que columnas son categóricas
print(f'El numero de columnas categoricas es de {Columnas_categoricas} y los nombres de estas columnas categoricas son {Nombre_columnas_categoricas}.')


El dataset debe tener un minimo de 4 columnas categoricas para cumplir con el requisito del proyecto.
El numero de columnas categoricas es de 8 y los nombres de estas columnas categoricas son ['Nationality', 'PreferredFoot', 'PreferredPositions', 'PlayerWorkRate', 'Nation', 'Nation_Position', 'Club', 'Club_Position'].


# Verificación de datos nulos.
Se hace el cálculo de cuantos y que porcentaje de datos nulos tiene cada columna del datset, para que, en caso de no cumplir el mínimo de 5% de datos nulos en la columna, se lleve a cabo el reemplazo de datos conocidos por datos nulos.

In [8]:
#Bucle for que asigna a cada columna su respectivo valor de datos nulos presnetes en ella misma
diccionario_valores_eliminar = {}
columnas_datos_nulos = []
porcentaje_datos_nulos ={}
for k in Datos_jugadores_fifa21.columns:
  
  if Datos_jugadores_fifa21[k].isnull().sum() !=0:
    print(f'La columna numero {(Datos_jugadores_fifa21.columns.tolist().index(k))+1} y de nombre {k} tiene',
    f'{Datos_jugadores_fifa21[k].isnull().sum()} datos nulos y un {round((Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100,2)} % de datos nulos respecto al total.')
    columnas_datos_nulos.append(k)
    porcentaje_datos_nulos[k]= round((Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100,2)
    if (Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100 <5 :
      print(f'La columna numero {(Datos_jugadores_fifa21.columns.tolist().index(k))+1} y de nombre {k} no cumple con el 5% minimo de datos nulos, entonces, ')
      print(f'le hace falta un {round(5-(Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100,2)} % de datos para alcanzar el minimo, lo cual quiere decir que')
      print(f'habrá que eliminar {int((5-(Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100)*Tamaño_Datos[0]/100)} datos del dataset para alcanzar el 5% de datos nulos')
      print() #Print vacio para dejar un espacio vacio entre los renglones de descripicion de diferentes columnas impresos en pantalla
      diccionario_valores_eliminar[k]= int((5-(Datos_jugadores_fifa21[k].isnull().sum()/Tamaño_Datos[0])*100)*Tamaño_Datos[0]/100)
      #Diccionario para poder almacenar los valores que le hacen falta a una columna para alcanzar el requiaisto minimo
      lista_azar = [0]*(diccionario_valores_eliminar[k]+Datos_jugadores_fifa21[k].isnull().sum()) 
      #Lista llena de ceros con longitud igual a datos faltantes de la columna
      import random
      for llenado in range(len(lista_azar)):
        lista_azar[llenado]= random.randint(0,5000)
      for x in lista_azar:
          if Datos_jugadores_fifa21[k][x]!= np.nan:
            Datos_jugadores_fifa21[k][x]= np.nan
          if Datos_jugadores_fifa21[k][x]==np.nan:
             continue
          if Datos_jugadores_fifa21[k].isnull().sum()>= 250:
             break
    else:
      print() #Print vacio para dejar un espacio vacio entre los renglones de descripicion de diferentes columnas impresos en pantalla



La columna numero 11 y de nombre Value tiene 250 datos nulos y un 5.0 % de datos nulos respecto al total.

La columna numero 12 y de nombre Nation tiene 4250 datos nulos y un 85.0 % de datos nulos respecto al total.

La columna numero 13 y de nombre Nation_Position tiene 4250 datos nulos y un 85.0 % de datos nulos respecto al total.



# Verificación de requisitos del dataset.
A continuación y mediante condicionales se hace la verificación de los requisitos del dataset (Tamaño,columnas categoricas,columnas con datos nulos, porcentaje de datos nulos en las columnas), en caso de faltar algun requisito se imprimirá en pantalla la condición que hace falta.

In [9]:
if Datos_jugadores_fifa21.size>=5000*30:
  print('Cumple con la dimension de dataset minima de 5000 datos con 30 columnas.')
 
  if Columnas_categoricas> Tamaño_Datos[1]*0.1:
    print('Cumple con el minimo de columnas categoricas segun su numero de columnas.')
    print(f'El dataset debe tener minimo {int(Tamaño_Datos[1]*0.1)} y tiene {Columnas_categoricas}')
    if len(columnas_datos_nulos)>=3:
      print(f'Cumple con el minimo de 3 columnas con datos nulos, tiene {columnas_datos_nulos} columnas con datos nulos.')
    
      for j,z in zip(porcentaje_datos_nulos.values(),porcentaje_datos_nulos.keys()):
        if j >=5:
          print(f'La columna {z} cumple con el minimo de 5% teniendo un {j} % de datos nulos')
        else:
          print(f'La columna {z} no cumple con el minimo de 5%, solo tiene un {j} % de datos nulos')
    else:
      print(f'El dataset no tiene suficientes columnas con datos nulos, solo tiene {len(columnas_datos_nulos)} ')
  else:
    print(f'El dataset no tiene el minimo de columnas categoricas, deberia tener {Tamaño_Datos[1]*0.1} columnas categoricas.')  
else:
  print(f'El dataset no cumple con el minimo de 5000 filas y 30 columnas, tiene {Tamaño_Datos[0]} filas y {Tamaño_Datos[1]} columnas') 
  
  

Cumple con la dimension de dataset minima de 5000 datos con 30 columnas.
Cumple con el minimo de columnas categoricas segun su numero de columnas.
El dataset debe tener minimo 4 y tiene 8
Cumple con el minimo de 3 columnas con datos nulos, tiene ['Value', 'Nation', 'Nation_Position'] columnas con datos nulos.
La columna Value cumple con el minimo de 5% teniendo un 5.0 % de datos nulos
La columna Nation cumple con el minimo de 5% teniendo un 85.0 % de datos nulos
La columna Nation_Position cumple con el minimo de 5% teniendo un 85.0 % de datos nulos


# Codigo usado para descargar el datsset procesado en las celdas anteriores y usarlo en otro notebook.
Esta celda no es necesaria que usted la ejecute, como se mencionó anteriormente, se usó para descargar el dataset procesado y posteriormente subirlo al repositorio del proyecto para que en proximos notebooks pueda ser descargado a colab desde el repositorio usando su url, tal como se importo el dataset original en la primera celda de este notebook.

In [35]:
Datos_jugadores_fifa21.to_csv('Datos_fifa21_procesado.csv')
from google.colab import files
files.download('Datos_fifa21_procesado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>